# Lambda School Data Science Module 141
## Statistics, Probability, and Inference

## Prepare - examine what's available in SciPy

As we delve into statistics, we'll be using more libraries - in particular the [stats package from SciPy](https://docs.scipy.org/doc/scipy/reference/tutorial/stats.html).

In [1]:
from scipy import stats
dir(stats)

['__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_binned_statistic',
 '_constants',
 '_continuous_distns',
 '_discrete_distns',
 '_distn_infrastructure',
 '_distr_params',
 '_multivariate',
 '_stats',
 '_stats_mstats_common',
 '_tukeylambda_stats',
 'absolute_import',
 'alpha',
 'anderson',
 'anderson_ksamp',
 'anglit',
 'ansari',
 'arcsine',
 'argus',
 'bartlett',
 'bayes_mvs',
 'bernoulli',
 'beta',
 'betaprime',
 'binned_statistic',
 'binned_statistic_2d',
 'binned_statistic_dd',
 'binom',
 'binom_test',
 'boltzmann',
 'boxcox',
 'boxcox_llf',
 'boxcox_normmax',
 'boxcox_normplot',
 'bradford',
 'burr',
 'burr12',
 'cauchy',
 'chi',
 'chi2',
 'chi2_contingency',
 'chisquare',
 'circmean',
 'circstd',
 'circvar',
 'combine_pvalues',
 'contingency',
 'cosine',
 'crystalball',
 'cumfreq',
 'describe',
 'dgamma',
 'dirichlet',
 'distributions',
 'division',
 'dlaplace',
 'dweibull',
 'energy_dist

In [2]:
# As usual, lots of stuff here! There's our friend, the normal distribution
norm = stats.norm()
print(norm.mean())
print(norm.std())
print(norm.var())

0.0
1.0
1.0


In [3]:
# And a new friend - t
t1 = stats.t(5)  # 5 is df "shape" parameter
print(t1.mean())
print(t1.std())
print(t1.var())

0.0
1.2909944487358056
1.6666666666666667


![T distribution PDF with different shape parameters](https://upload.wikimedia.org/wikipedia/commons/4/41/Student_t_pdf.svg)

*(Picture from [Wikipedia](https://en.wikipedia.org/wiki/Student's_t-distribution#/media/File:Student_t_pdf.svg))*

The t-distribution is "normal-ish" - the larger the parameter (which reflects its degrees of freedom - more input data/features will increase it), the closer to true normal.

In [4]:
t2 = stats.t(30)  # Will be closer to normal
print(t2.mean())
print(t2.std())
print(t2.var())

0.0
1.0350983390135313
1.0714285714285714


Why is it different from normal? To better reflect the tendencies of small data and situations with unknown population standard deviation. In other words, the normal distribution is still the nice pure ideal in the limit (thanks to the central limit theorem), but the t-distribution is much more useful in many real-world situations.

History sidenote - this is "Student":

![William Sealy Gosset](https://upload.wikimedia.org/wikipedia/commons/4/42/William_Sealy_Gosset.jpg)

*(Picture from [Wikipedia](https://en.wikipedia.org/wiki/File:William_Sealy_Gosset.jpg))*

His real name is William Sealy Gosset, and he published under the pen name "Student" because he was not an academic. He was a brewer, working at Guinness and using trial and error to determine the best ways to yield barley. He's also proof that, even 100 years ago, you don't need official credentials to do real data science!

## Live Lecture - let's perform and interpret a t-test

We'll generate our own data, so we can know and alter the "ground truth" that the t-test should find. We will learn about p-values and how to interpret "statistical significance" based on the output of a hypothesis test.

In [0]:
# TODO - during class, but please help!

## Assignment - apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data' 
names = ['party','handicapped-infants','water-project-cost-sharing','adoption-of-the-budget-resolution','physician-fee-freeze','el-salvador-aid','religious-groups-in-schools','anti-satellite-test-ban','aid-to-nicaraguan-contras','mx-missile','immigration','synfuels-corporation-cutback','education-spending','superfund-right-to-sue','crime','duty-free-exports','export-administration-act-south-africa']
issues = names[1:]
roster = pd.read_csv(url, na_values='?', names=names)
roster.replace(['y','n'],[1,0], inplace=True)

reps = roster[roster.party == 'republican']
dems = roster[roster.party == 'democrat']

## First iteration

In [109]:
dem_favs = []
rep_favs = []
bipartisan = []
other = []

print('{:40} {:<10} {}'.format('Issue', 'Statistic', 'p_value'))
print('-'*60)

for issue in issues:    
    stat, p_value = stats.ttest_ind(dems[issue], reps[issue],
                                    nan_policy='omit', equal_var=False)
    print('{:40} {:<10.3} {:.3}'.format(issue, stat, p_value))

    if p_value < 0.01 and stat > 0:
        dem_favs.append(issue)
    elif p_value < 0.01 and stat < 0:
        rep_favs.append(issue)
    elif p_value > 0.1:
        bipartisan.append(issue)
    else:
        other.append(issue)

Issue                                    Statistic  p_value
------------------------------------------------------------
handicapped-infants                      9.66       5.43e-20
water-project-cost-sharing               -0.0889    0.929
adoption-of-the-budget-resolution        22.8       1.95e-69
physician-fee-freeze                     -56.7      8.82e-193
el-salvador-aid                          -23.9      2.9e-79
religious-groups-in-schools              -10.8      4.83e-24
anti-satellite-test-ban                  12.5       1.36e-29
aid-to-nicaraguan-contras                18.3       2.29e-52
mx-missile                               17.4       4.29e-51
immigration                              -1.74      0.0832
synfuels-corporation-cutback             9.04       6.3e-18
education-spending                       -20.7      3.57e-62
superfund-right-to-sue                   -14.4      7.6e-38
crime                                    -19.9      6.86e-58
duty-free-exports               

In [110]:
# Issues favored by democrats, p < 0.01
dem_favs

['handicapped-infants',
 'adoption-of-the-budget-resolution',
 'anti-satellite-test-ban',
 'aid-to-nicaraguan-contras',
 'mx-missile',
 'synfuels-corporation-cutback',
 'duty-free-exports',
 'export-administration-act-south-africa']

In [111]:
# Issues favored by republicans, p < 0.01
rep_favs

['physician-fee-freeze',
 'el-salvador-aid',
 'religious-groups-in-schools',
 'education-spending',
 'superfund-right-to-sue',
 'crime']

In [112]:
# Issues favored by neither (P > 0.1)
bipartisan

['water-project-cost-sharing']

In [113]:
other

['immigration']

## Stretch goal!  Refactoring the code above into functions

In [208]:
def partisan_printout(roster, significance_cutoff=0.01,
                     bipartisan_cutoff=0.1):
    """
    Takes in a dataframe of congressional voting and identifies the
    issues on which there was a significant voting difference between
    Republicans and Democrats. Comparison is made using Welch's t-test,
    which does not assume equal variances or sample sizes.
    
    Input parameters:
    
    roster
        A dataframe in which the column 'party' contains
        classifies the congressperson as 'republican' or 'democrat',
        the other columns are names of issues,
        and each cell contains 0 (no vote), 1 (yes vote), or NaN.

    significance_cutoff
        Maximum p-value (non-inclusive) for an issue to be considered
        significantly partisan
        
    bipartisan_cutoff
        Minimum p-value (non-inclusive) for an issue to be considered
        bipartisan
    
    Output:
    
    Printout with all the issues, classified by partisan preference.
    """
    reps = roster[roster.party == 'republican']
    dems = roster[roster.party == 'democrat']
    issues = roster.columns[1:]

    dem_favs = []
    rep_favs = []
    bipartisan = []
    other = []
    
#   I populate the lists above with the issues that fall in each category. Each issue is a tuple of 
#   name, state, and p_value
    for issue in issues:    
        # Calculates Welch's t-test for each issue, comparing Dems to Reps.
        stat, p_value = stats.ttest_ind(dems[issue], reps[issue],
                                        nan_policy='omit', 
                                        equal_var=False)

        
        if p_value < significance_cutoff and stat > 0:
            dem_favs.append((issue, stat, p_value))
            
        elif p_value < significance_cutoff and stat < 0:
            rep_favs.append((issue, stat, p_value))
            
        elif p_value > bipartisan_cutoff:
            bipartisan.append((issue, stat, p_value))
            
        else:
            other.append((issue, stat, p_value))
            
            
    # This block is for printing out the contents of all the categories above
    classifications = [dem_favs, rep_favs, bipartisan, other]
    headers = ['Issues favored by Democrats, p<{}',
                'Issues favored by Republicans, p<{}',
                'Bipartisan Issues, p>{}',
                'Other Issues']
    cutoffs = [significance_cutoff, significance_cutoff, bipartisan_cutoff, 0]
    
    for header, classification, cutoff in zip(headers, classifications, cutoffs):
        print('{:50} {:<20} {}'.format('\n\n'+header.format(cutoff), "Welch's t-test", 'p_value'))
        print('-'*80)
    
        for issue in classification:
            print('{:50} {:<18.3} {:.3}'.format(issue[0], issue[1], issue[2]))
        
        
        
        

In [209]:
partisan_printout(roster, 0.01, 0.1)



Issues favored by Democrats, p<0.01              Welch's t-test       p_value
--------------------------------------------------------------------------------
handicapped-infants                                9.66               5.43e-20
adoption-of-the-budget-resolution                  22.8               1.95e-69
anti-satellite-test-ban                            12.5               1.36e-29
aid-to-nicaraguan-contras                          18.3               2.29e-52
mx-missile                                         17.4               4.29e-51
synfuels-corporation-cutback                       9.04               6.3e-18
duty-free-exports                                  14.4               3.81e-38
export-administration-act-south-africa             6.4                1.04e-09


Issues favored by Republicans, p<0.01            Welch's t-test       p_value
--------------------------------------------------------------------------------
physician-fee-freeze                           